In [1]:
pip install music21

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import glob
import pickle
import numpy
import os
from sklearn.model_selection import train_test_split
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
import keras.utils
from keras.callbacks import ModelCheckpoint

In [17]:
notes_train = []
durations_train = [] 

train_song = {}

dir_path = "dataset"
for folder in os.listdir(dir_path):
    train_song[str(folder)] = folder

In [29]:
folder = os.listdir(dir_path)

for fold in folder:
      try:
          midi = converter.parse(f"dataset/{fold}")
          
          prev_offset = 0.0

          for element in midi.flat.notes:
            if isinstance(element, note.Note):
              notes_train.append(str(element.pitch))
              duration = element.duration.quarterLength
              durations_train.append(duration)
            elif isinstance(element, chord.Chord):
              notes_train.append(".".join(str(n) for n in element.normalOrder))
              duration = element.duration.quarterLength
              durations_train.append(duration)
              prev_offset = element.offset + element.duration.quarterLength
      
      except Exception as e:
          print(f"Error processing file: {fold} - {e}")
          pass

Error processing file: ff8compressionoftime.mid - badly formed midi string: missing leading MTrk


c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\music21\midi\translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Final Fantasy \x87[\x81@\x81@\x81u\x8e\x80\x82\xcc\x93\xa6\x91\x96\x81i\x89\xbc\x81j\x81v'>; getting generic Instrument
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\music21\midi\translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Final Fantasy VIII \x81` The Extreme \x81`  by QMA'>; getting generic Instrument
  warnings.warn(


Error processing file: ff8liberifatali.mid - badly formed midi string: missing leading MTrk


c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\music21\midi\translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'FF\x87[  [\x83o\x83g\x83\x8b\x82P\x81i\x89\xbc\x81j]  for SC-88  -NEGI-'>; getting generic Instrument
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\music21\midi\translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'\x90\xed\x93\xac\x83V\x81[\x83\x93\x81i\x89\xbc\x81jfrom FF8\x91\xcc\x8c\xb1\x94\xc5'>; getting generic Instrument
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python39\lib\site-packages\music21\midi\translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Final Fantasy VIII \x81`A Sacrifaice\x81`  <88> by

In [30]:
notes_train

['A3',
 'E4',
 'C#5',
 'B4',
 'D5',
 'E5',
 'A4',
 'G#4',
 'G#5',
 'F#5',
 'A3',
 'D4',
 'D5',
 'F#4',
 'B4',
 'D5',
 'B-4',
 'D4',
 'A3',
 'E4',
 'B4',
 'G#4',
 'C#5',
 'A4',
 'D5',
 'B4',
 'E4',
 'B3',
 'F#5',
 'E-5',
 'A3',
 'E4',
 'E5',
 'A4',
 'B4',
 'E5',
 'C#5',
 'B4',
 'A4',
 'E4',
 'A3',
 'E4',
 'C#5',
 'B4',
 'D5',
 'E4',
 'E5',
 'A4',
 'E4',
 'G#4',
 'G#5',
 'E4',
 'F#5',
 'A3',
 'F#4',
 'D5',
 'C#5',
 'F#4',
 'D5',
 'B4',
 'F#4',
 'A4',
 'F#4',
 'E3',
 'E4',
 'B4',
 'G#4',
 'C#5',
 'A4',
 'D5',
 'B4',
 'E4',
 'B3',
 'G#4',
 'B4',
 'A3',
 'E4',
 'A4',
 'E4',
 'A4',
 'E4',
 'E4',
 'D4',
 'A4',
 'D5',
 'E5',
 'C#5',
 'F#5',
 'B4',
 'A4',
 'A5',
 'F#4',
 'A5',
 'C#4',
 'E4',
 'C#5',
 'D5',
 'B4',
 'E5',
 'A4',
 'G#4',
 'A5',
 'F#4',
 'E4',
 'A5',
 'B3',
 'F#4',
 'B4',
 'C#5',
 'A4',
 'D5',
 'B4',
 'A4',
 'F#5',
 'F#4',
 'D4',
 'E-5',
 'A3',
 'E4',
 'E5',
 'A4',
 'B4',
 'C#6',
 'A4',
 'C#5',
 'A4',
 'E4',
 'D4',
 'A4',
 'D5',
 'E5',
 'C#5',
 'F#5',
 'B4',
 'A4',
 'A5',
 'F#4',
 

In [ ]:
with open('notes_train.txt', 'w') as notes_file:
    for note in notes_train:
        notes_file.write(f"{note}\n")

with open('durations_train.txt', 'w') as durations_file:
    for duration in durations_train:
        durations_file.write(f"{duration}\n")